## Building Tables for AWS RDS PostgresSQL

In [5]:
import psycopg2
import pandas as pd

In [6]:
connection = psycopg2.connect(
    host = 'project3.cdr354qikgy9.us-east-2.rds.amazonaws.com',
    port = 5431,
    user = 'NBA',
    password = 'NBAFun23!',
    database='NBA_predictor'
    )

#cursor object interacts and executes the commands on the Postgres database 
cursor=connection.cursor()

In [60]:
#check what is in the database
query = """SELECT * FROM pg_catalog.pg_tables
            WHERE schemaname != 'pg_catalog'
            AND schemaname != 'information_schema';"""
cursor.execute(query)
cursor.fetchall()

[('public', 'playerusage', 'NBA', None, False, False, False, False),
 ('public', 'boxscore', 'NBA', None, False, False, False, False),
 ('public', 'advcstats', 'NBA', None, False, False, False, False)]

## Creating Tables for Advanced Stats, Encoded Box Scores, Player Usage

In [24]:
AdvancedStats_table  = """CREATE TABLE AdvcStats(id integer, Player varchar(50), Pos varchar(5), Age integer, Tm varchar(5), 
                        G integer, MP integer, PER numeric, TSpct numeric, ThreePAr numeric, FTr numeric,
                        ORBpct numeric, DRBpct numeric, TRBpct numeric, ASTpct numeric, STLpct numeric, 
                        BLKpct numeric, TOVpct numeric, USGpct numeric, OWS numeric, DWS numeric, 
                        WS numeric, WS48 numeric, OBPM numeric, BPM numeric, VORP numeric, Year integer, 
                        slug varchar(20));""";

cursor.execute(AdvancedStats_table);
#Commit table creation
connection.commit()

#### importing advanced stats .csv file to table 

In [27]:
#inserting .csv files using copy_from() into table  
with open('../Resources/adv_stats_all_slug.csv', 'r', encoding="utf8") as row:
    # Skip the header row
    next(row)
    cursor.copy_from(row, 'AdvcStats', sep=',')
connection.commit()

In [29]:
connection.rollback()

In [30]:
#Test the data was loaded 
sql = """
SELECT * from AdvcStats 
WHERE player = 'Quincy Acy'
"""

pd.read_sql(sql, con=connection)

,id,player,pos,age,tm,g,mp,per,tspct,threepar,...,usgpct,ows,dws,ws,ws48,obpm,bpm,vorp,year,slug
0,0,Quincy Acy,PF,22,TOR,29,342,15.9,0.632,0.027,...,14.7,0.7,0.4,1.1,0.157,-0.6,0.7,0.2,2013,acyqu01
1,471,Quincy Acy,SF,23,TOT,63,847,10.1,0.520,0.106,...,10.1,0.6,0.9,1.5,0.086,-2.5,-1.1,0.2,2014,acyqu01
2,956,Quincy Acy,PF,24,NYK,68,1287,11.9,0.533,0.181,...,15.5,1.0,0.7,1.7,0.063,-2.3,-3.1,-0.3,2015,acyqu01
3,1449,Quincy Acy,PF,25,SAC,59,876,14.7,0.629,0.229,...,13.1,1.8,0.7,2.5,0.137,0.2,0.2,0.5,2016,acyqu01
4,1927,Quincy Acy,PF,26,TOT,38,558,11.8,0.565,0.529,...,16.8,0.5,0.5,0.9,0.082,-1.8,-3.0,-0.1,2017,acyqu01
5,2413,Quincy Acy,PF,27,BRK,70,1359,8.2,0.525,0.800,...,14.4,-0.1,1.1,1.0,0.036,-2.0,-2.2,-0.1,2018,acyqu01
6,2954,Quincy Acy,PF,28,PHO,10,123,2.9,0.379,0.833,...,9.2,-0.1,0.0,-0.1,-0.022,-5.7,-5.9,-0.1,2019,acyqu01


In [56]:
cursor.execute("""DROP TABLE allusage""")
connection.commit()

In [21]:
Encoded_Box_ScoresTable =    """CREATE TABLE boxscore(id integer, slug varchar(20), name varchar(40), Team varchar(40), Location varchar(6),
                                Opponent varchar(40), Outcome varchar(5), seconds_played numeric, made_field_goals numeric, atempted_filed_goals numeric, 
                                made_three_point_fg numeric, att_three_point_fg numeric, made_free_throws numeric, attempted_free_throws numeric, 
                                offensive_rebounds numeric, defensive_rebounds numeric, assists numeric, steals numeric, blocks numeric, turnovers numeric,
                                personal_fouls numeric, game_score numeric, date date, game_id integer, spread integer);""";

cursor.execute(Encoded_Box_ScoresTable);
connection.commit()

In [22]:
#inserting .csv files using copy_from() into table  
with open('../Resources/encoded_box_scores.csv', 'r', encoding="utf8") as row:
    # Skip the header row
    next(row)
    cursor.copy_from(row, 'boxscore', sep=',')
connection.commit()

In [24]:
sql = """
SELECT * from boxscore
"""

pd.read_sql(sql, con=connection)

,id,slug,name,team,location,opponent,outcome,seconds_played,made_field_goals,atempted_filed_goals,...,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,game_score,date,game_id,spread
0,0,anthoca01,Carmelo Anthony,NEW_YORK_KNICKS,HOME,PORTLAND_TRAIL_BLAZERS,LOSS,2472.0,14.0,24.0,...,5.0,4.0,1.0,0.0,2.0,3.0,36.5,2013-01-01,459,5
1,1,smithjr01,J.R. Smith,NEW_YORK_KNICKS,HOME,PORTLAND_TRAIL_BLAZERS,LOSS,2413.0,10.0,22.0,...,8.0,5.0,3.0,0.0,2.0,2.0,24.0,2013-01-01,459,5
2,2,bryanko01,Kobe Bryant,LOS_ANGELES_LAKERS,HOME,PHILADELPHIA_76ERS,LOSS,2545.0,14.0,29.0,...,2.0,0.0,1.0,0.0,3.0,2.0,20.7,2013-01-01,462,4
3,3,anderry01,Ryan Anderson,NEW_ORLEANS_HORNETS,HOME,ATLANTA_HAWKS,LOSS,1928.0,9.0,15.0,...,3.0,0.0,0.0,0.0,0.0,2.0,18.3,2013-01-01,460,9
4,4,vasqugr01,Greivis Vásquez,NEW_ORLEANS_HORNETS,HOME,ATLANTA_HAWKS,LOSS,2317.0,7.0,15.0,...,8.0,10.0,0.0,1.0,2.0,1.0,16.9,2013-01-01,460,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200892,100377,netora01,Raul Neto,PHILADELPHIA_76ERS,AWAY,MILWAUKEE_BUCKS,LOSS,23.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-02-06,9958,11
200893,100378,smithzh01,Zhaire Smith,PHILADELPHIA_76ERS,AWAY,MILWAUKEE_BUCKS,LOSS,23.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-02-06,9958,11
200894,100379,belinma01,Marco Belinelli,SAN_ANTONIO_SPURS,AWAY,PORTLAND_TRAIL_BLAZERS,LOSS,166.0,0.0,2.0,...,1.0,1.0,0.0,0.0,0.0,0.0,-0.4,2020-02-06,9959,8
200895,100380,okafoja01,Jahlil Okafor,NEW_ORLEANS_PELICANS,AWAY,CHICAGO_BULLS,WIN,423.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,1.0,-0.4,2020-02-06,9957,6


In [57]:
AllUsageTable = """CREATE TABLE playerusage(ID integer, GP numeric, MIN numeric, USGper numeric, FGMper numeric, FGAper numeric, threePMper numeric,
                    threePAper numeric, FTMper numeric, FTAper numeric, OREBper numeric, DREBper numeric, REBper numeric, ASTper numeric, TOVper numeric,
                    STLper numeric, BLKper numeric, BLKAper numeric, PFper numeric, PFDper numeric, PTSper numeric, stats_id integer, 
                    player varchar(40), year integer, number integer, slug varchar(25));""";

cursor.execute(AllUsageTable);
connection.commit()

In [58]:
with open('../Resources/all_usage_clean.csv', 'r', encoding="utf8") as row: 
    next(row)
    cursor.copy_from(row, 'playerusage', sep=',')
connection.commit()

In [59]:
#Test the data was loaded 
sql = """
SELECT * from playerusage 
WHERE player = 'Quincy Acy'
"""

pd.read_sql(sql, con=connection)

,id,gp,min,usgper,fgmper,fgaper,threepmper,threepaper,ftmper,ftaper,...,blkper,blkaper,pfper,pfdper,ptsper,stats_id,player,year,number,slug
0,477,63.0,848.0,9.9,10.7,9.8,3.4,4.3,9.0,10.3,...,33.3,10.0,27.1,12.4,9.8,203112,Quincy Acy,2014,0,acyqu01
1,926,68.0,1287.0,15.1,16.9,15.2,9.0,10.5,20.9,20.6,...,18.6,10.3,26.5,22.6,16.8,203112,Quincy Acy,2015,1,acyqu01
2,1363,59.0,876.0,12.8,16.7,13.5,13.3,11.8,19.2,18.3,...,28.9,6.5,28.1,20.6,16.8,203112,Quincy Acy,2016,2,acyqu01
3,1791,38.0,558.0,16.5,17.1,17.7,29.1,24.9,17.2,18.6,...,32.6,10.0,25.8,22.8,18.4,203112,Quincy Acy,2017,3,acyqu01
4,2265,70.0,1359.0,14.3,12.3,15.3,25.6,26.5,10.3,10.0,...,23.8,9.5,24.5,14.7,13.7,203112,Quincy Acy,2018,4,acyqu01
5,2731,10.0,123.0,8.8,4.2,8.2,7.4,16.1,12.3,12.8,...,25.0,0.0,31.6,21.2,6.2,203112,Quincy Acy,2019,5,acyqu01


In [48]:
connection.rollback()